In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle
import scipy.sparse as sp
import os

HGCN_HOME= os.getcwd()
print(HGCN_HOME)

In [ ]:
import train
import args
from config import parser
from utils.data_utils import load_data, load_inference_data, clean_and_process_nexus
from utils.tree.load_parameters import generate_data
%load_ext autoreload
%autoreload 2

### Import libraries

In [ ]:
from utils.tree.load_parameters import generate_data, get_labels
from utils.tree.load_parameters import one_hot
from utils.tree.data_utils import simulate_seq, simulate_seq_all
from utils.tree.tree_utils import create_tree
from utils.tree.sem import randomt_tree
import utils.tree.computations as cmp
from utils.tree.load_parameters import build_weighted_adjacency_matrix
from utils.tree.substitution_models import JukesCantor
from utils.tree.mst_utils import bifurcate_mst, get_mst

import train
import args
from config import parser

%load_ext autoreload
%autoreload 2

#### Function to generate interiior nodes given tree structure and leaf sequences

In [66]:
from collections import deque
import numpy as np
import networkx as nx

def generate_interior_node(tree, leaf_features, evo_model):
    n_nodes = len(tree)
    n_leaves = len(leaf_features)

    # Initialize node features for all nodes (only leaf nodes have initial features)
    node_features = {i: leaf_features[i] for i in range(n_leaves)}
    for i in range(n_leaves, n_nodes):
        node_features[i] = np.zeros(leaf_features.shape[1])

    # Create a parent map to keep track of parent-child relationships
    parent_map = {}
    root = max(tree.nodes)

    # Assuming the tree is undirected, build parent-child relationships to identify tree structure
    stack = [root]
    visited = set()
    visited.add(root)

    while stack:
        node = stack.pop()
        children = list(tree.neighbors(node))
        for child in children:
            if child not in visited:
                parent_map[child] = node
                stack.append(child)
                visited.add(child)

    # Postorder Traversal to Collect Parameters (Iterative version)
    c_params = np.zeros(n_nodes)
    d_params = np.zeros((n_nodes, leaf_features.shape[1]))
    visited.clear()

    # Using a stack for iterative postorder traversal
    stack = deque()
    stack.append((root, False))  # Start from the root

    while stack:
        node, processed = stack.pop()

        if processed:
            # Process the node after its children have been processed
            if node not in visited:
                children = [child for child in tree.neighbors(node) if child != parent_map.get(node)]
                for child in children:
                    c_params[node] += 1 + c_params[child]
                    d_params[node] += d_params[child]
                visited.add(node)
        else:
            # Push the node back to be processed after its children
            stack.append((node, True))
            # Push all children to be processed first
            children = [child for child in tree.neighbors(node) if child != parent_map.get(node)]
            for child in children:
                if child not in visited:
                    stack.append((child, False))

    # Preorder Traversal to Generate Features for Interior Nodes (Iterative version)
    stack = deque([root])  # Start from the root
    visited.clear()

    while stack:
        node = stack.pop()
        if node not in visited:
            # Generate node feature for the root or other nodes
            if node == root:
                node_features[node] = d_params[node] / (1 + c_params[node])
            else:
                parent_node = parent_map[node]
                node_features[node] = (c_params[node] * node_features[parent_node] + d_params[node]) / (c_params[node] + 1)

            # Mark the node as visited
            visited.add(node)

            # Add children to the stack
            children = [child for child in tree.neighbors(node) if child != parent_map.get(node)]
            for child in children:
                if child not in visited:
                    stack.append(child)

    # Convert the node features to a numpy array
    all_sequences = []
    for i in range(n_nodes):
        all_sequences.append(node_features[i])
    all_sequences_np = np.array(all_sequences)

    # Create Adjacency Matrix
    adj_matrix = np.zeros((n_nodes, n_nodes), dtype=int)
    for edge in tree.edges:
        parent, child = edge
        adj_matrix[parent, child] = 1
        adj_matrix[child, parent] = 1  # Assuming undirected tree

    return all_sequences_np, adj_matrix

In [67]:
import torch

def predict_weighted_adj_matrix(leaf_features, tree, model):
    
    feature_tensor = torch.tensor(leaf_features, dtype=torch.float)
    print(feature_tensor.shape)
    
    # Convert adjacency matrix to a PyTorch sparse tensor
    G = nx.complete_graph(len(tree))
    adj = nx.to_scipy_sparse_matrix(G, format='csr')
    adj_coo = adj.tocoo()
    indices = torch.from_numpy(np.vstack((adj_coo.row, adj_coo.col)).astype(np.int64))
    print(indices.shape)

    values = torch.from_numpy((adj_coo.data).astype(np.float32))
    print(values.dtype)

    shape = torch.Size(adj_coo.shape)
    adj_tensor = torch.sparse.FloatTensor(indices, values, shape)
    print(adj_tensor.dtype)

    with torch.no_grad():
        embeddings = model.encode(feature_tensor, adj_tensor)
        link_probabilities = model.decode(embeddings, indices.T)

    link_pred_np = np.array(link_probabilities)
    n_nodes = adj.shape[0]
    weighted_adj_matrix = np.zeros((n_nodes, n_nodes))
    # Populate the adjacency matrix based on link_predictions
    for i in range(link_pred_np.shape[0]):
        u, v, weight = int(indices[0,i]), int(indices[1,i]), link_pred_np[i]
        weighted_adj_matrix[u, v] = weight
        weighted_adj_matrix[v, u] = weight 

    return weighted_adj_matrix


### Loop 1

- train (e.g. 50) sequences first
- predict on test data

In [194]:
def train_loop1(leaf_sequences, evo_model, model):
    """
        leaf_sequences: nucleotide (ATCG)

    """
    # encode the leaf sequences
    encoded_seq = one_hot(leaf_sequences)
    leaf_features = np.argmax(encoded_seq, axis=-1)
    
    n_leaves, n_sites = leaf_sequences.shape
    # construct a initial random tree ontology
    tree, opts = randomt_tree(n_leaves)
    
    leaves = [n for n in tree if tree.nodes[n]['type'] == 'leaf']
    root = len(tree) - 1    
    
    sigma_l = 0.1  # annealing std
    rho = 0.95  # cooling factor
    max_iter = 100
    T_l = tree  # init topology
    ll_vec = []  # log-likelihood
    Tl_vec=[]
    max_iter = 20
    
   
    for iter in range(max_iter):
        print(f'start iter: {iter}')
        
        Tl_vec.append(tree)
        up_table = cmp.compute_up_messages(leaf_sequences, tree, evo_model)
        down_table = cmp.compute_down_messages(leaf_sequences, tree, evo_model, up_table)
        # compute log-likelihood
        ll_sites, ll = cmp.compute_loglikelihood(up_table, evo_model.stat_prob)
        if len(ll_vec)>0 and ll_vec[0]>ll:#figure out argmax maybe
            tree = Tl_vec[0]
            ll_vec.append(ll_vec[0])
            continue
        ll_vec.append(ll)
        print(f'log-likelihood: {ll}')

        W, opts = cmp.compute_weight_matrix(tree, evo_model, up_table, down_table)
        # print("computed weighted matrix")

        # generate interior nodes and (unweighted) adj matrix sequences given leaf sequences
        features, adj_matrix = generate_interior_node(tree, leaf_features, evo_model)
        print("generated interior nodes")
       
        ## link-prediction
        weighted_adj_matrix = predict_weighted_adj_matrix(features, tree, model)

        # construct mst from weighted adj matrix
        mst = get_mst(weighted_adj_matrix, opts)
        # construct bifurcate tree from mst, as the tree onto for next iter
        tree, edge = bifurcate_mst(mst, leaves, root)

        sigma_l *= rho
        if sigma_l <= 5e-3:
            break
        elif iter > 0 and np.abs(ll_vec[iter] - ll_vec[iter - 1]) < 1e-3:
            break

        print(f'finished iter: {iter}')

    return tree, ll_vec
        

In [ ]:
args.args.manifold

In [ ]:
import train
import args
from config import parser

number = 1748
# train the model on 50 simulated sequences
filepath = f"/Users/liubojun/Desktop/hgcn/data/inference/M{number}.nex"
seq = clean_and_process_nexus(filepath)

num_data = 50
n_leaves, n_features = seq.shape
model = train.train_multiple(args.args, num_data, n_leaves, n_features)

In [ ]:
import torch

model_path = "model.pth"

torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


In [ ]:
evo_model = JukesCantor(alpha=0.1)

tree, ll_vec = train_loop1(seq, evo_model, model)

### Loop 2

In [216]:
def train_loop2(leaf_sequences, evo_model):
    """
        leaf_sequences: nucleotide (ATCG)

    """
    # encode the leaf sequences
    encoded_seq = one_hot(leaf_sequences)
    leaf_features = np.argmax(encoded_seq, axis=-1)
    
    n_leaves, n_features = leaf_sequences.shape
    # construct a initial random tree ontology
    tree, opts = randomt_tree(n_leaves)
    
    leaves = [n for n in tree if tree.nodes[n]['type'] == 'leaf']
    root = len(tree) - 1    
    
    sigma_l = 0.1  # annealing std
    rho = 0.95  # cooling factor
    max_iter = 100
    T_l = tree  # init topology
    ll_vec = []  # log-likelihood
    Tl_vec=[]
    max_iter = 100
    
    print("start training loop")
    for iter in range(max_iter):
        print(f'start iter: {iter}')
        
        Tl_vec.append(tree)
        up_table = cmp.compute_up_messages(leaf_sequences, tree, evo_model)
        down_table = cmp.compute_down_messages(leaf_sequences, tree, evo_model, up_table)
        # compute log-likelihood
        ll_sites, ll = cmp.compute_loglikelihood(up_table, evo_model.stat_prob)
        if len(ll_vec)>0 and ll_vec[0]>ll:#figure out argmax maybe
            tree = Tl_vec[0]
            ll_vec.append(ll_vec[0])
            continue
        ll_vec.append(ll)
        print(f'log-likelihood: {ll}')

        W, opts = cmp.compute_weight_matrix(tree, evo_model, up_table, down_table)
        # print("computed weighted matrix")

        # generate interior nodes and (unweighted) adj matrix sequences given leaf sequences
        features, adj_matrix = generate_interior_node(tree, leaf_features, evo_model)
        print("generated interior nodes")
        
        # train 
        adj = sp.csr_matrix(adj_matrix) # scipy sparce matrix
        labels = get_labels(tree)
        data = load_inference_data(adj, features, labels, opts, args.args)
        print("start training")
        weighted_adj_matrix, labels, opts, model = train.train(args.args, data, n_features)
        print("finished training")
        # construct mst from weighted adj matrix
        mst = get_mst(weighted_adj_matrix, opts)
        # construct bifurcate tree from mst, as the tree onto for next iter
        tree, edge = bifurcate_mst(mst, leaves, root)

        sigma_l *= rho
        if sigma_l <= 5e-3:
            break
        elif iter > 0 and np.abs(ll_vec[iter] - ll_vec[iter - 1]) < 1e-3:
            break

        print(f'finished iter: {iter}')

    return tree, ll_vec
        

In [ ]:
# 1310, 1366, 1367, 1745, 1746, 1747, 1748, 1749
numbers = [1310, 1366, 1367, 1745, 1746, 1747, 1748, 1749]

for number in numbers:
    filepath = f"/Users/liubojun/Desktop/hgcn/data/inference/M{number}.nex"
    seq = clean_and_process_nexus(filepath)

    # Euclidean
    args.args.manifold = 'Euclidean'
    manifold = args.args.manifold
    evo_model = JukesCantor(alpha=0.1)
    tree, ll_vec_euc = train_loop2(seq, evo_model)

    # Save the NumPy array as a FASTA file
    save_as_fasta(seq, f"results/M{number}/{manifold}/M{number}.fasta")
    # save tree
    with open(f"results/M{number}/{manifold}/pred_tree_M{number}.pkl", "wb") as f:
        pickle.dump(tree, f)
    # save log-likelihood
    np.savetxt(f'results/M{number}/{manifold}/ll_vec_M{number}.txt',ll_vec_euc)

    # Hyperbolic
    args.args.manifold = 'PoincareBall'
    manifold = args.args.manifold
    evo_model = JukesCantor(alpha=0.1)
    tree, ll_vec_hyp = train_loop2(seq, evo_model)

    # Save the NumPy array as a FASTA file
    save_as_fasta(seq, f"results/M{number}/{manifold}/M{number}.fasta")
    # save tree
    with open(f"results/M{number}/{manifold}/pred_tree_M{number}.pkl", "wb") as f:
        pickle.dump(tree, f)
    # save log-likelihood
    np.savetxt(f'results/M{number}/{manifold}/ll_vec_M{number}.txt',ll_vec_hyp)

    
    fig, ax = plt.subplots()
    ax.plot(ll_vec_hyp)
    ax.plot(ll_vec_euc)
    ax.set(xlabel='iter', ylabel='log-likelihood', title=f"log-likelihood for M{number}")
    ax.grid()
    ax.legend(["hyperbolic", "euclidean"])
    fig.savefig(f"results/M{number}/ll_plot_M{number}.png") 
        

In [205]:
# 1310, 1366, 1367, 1745, 1746, 1747, 1748, 1749
numbers = [1310, 1366, 1367, 1745, 1746, 1747, 1748, 1749]

for number in numbers:
    filepath = f"/Users/liubojun/Desktop/hgcn/data/inference/M{number}.nex"
    seq = clean_and_process_nexus(filepath)

    # Euclidean
    args.args.manifold = 'Euclidean'
    manifold = args.args.manifold

    # Save the NumPy array as a FASTA file
    save_as_fasta(seq, f"results/M{number}/{manifold}/M{number}.fasta")

    # Hyperbolic
    args.args.manifold = 'PoincareBall'
    manifold = args.args.manifold
    
    # Save the NumPy array as a FASTA file
    save_as_fasta(seq, f"results/M{number}/{manifold}/M{number}.fasta")

In [ ]:
fig, ax = plt.subplots()
ax.plot(ll_vec_hyp)
ax.plot(ll_vec_euc)
ax.set(xlabel='iter', ylabel='log-likelihood', title=f"log-likelihood for M{number}")
ax.grid()
ax.legend(["hyperbolic", "euclidean"])
fig.savefig(f"results/M{number}/ll_plot_M{number}.png") 


### PoincareBall

In [173]:
args.args.manifold = 'Euclidean'

In [ ]:
manifold = args.args.manifold
manifold

In [ ]:
# n_leaves = 50
# tree, opts = randomt_tree(n_leaves)
# sim_seq = simulate_seq_all(tree, evo_model, ndata=len(tree))

evo_model = JukesCantor(alpha=0.1)
tree, ll_vec = train_loop2(seq, evo_model)

#### Save leaf sequences, predicted tree, and log-likelihood 

In [176]:
manifold = args.args.manifold

# convert to fasta
def save_as_fasta(sequences, file_name):
    directory = os.path.dirname(file_name)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        
    with open(file_name, 'w') as f:
        for i, seq in enumerate(sequences):
            # Write the header line
            f.write(f">Sequence_{i+1}\n")
            # Write the sequence line (convert array to a single string)
            f.write("".join(seq) + "\n")

# Save the NumPy array as a FASTA file
save_as_fasta(sim_seq, f"results/M{number}/{manifold}/M{number}.fasta")


import pickle

with open(f"results/M{number}/{manifold}/pred_tree_M{number}.pkl", "wb") as f:
    pickle.dump(tree, f)

# Load the saved NetworkX graph from the pickle file
# with open("mst_tree.pkl", "rb") as f:
#     mst_loaded = pickle.load(f)

# save log-likelihood
np.savetxt(f'results/M{number}/{manifold}/ll_vec_M{number}.txt',ll_vec)

In [177]:
ll_vec_euc = ll_vec

In [ ]:
plt.plot(ll_vec_hyp)
plt.plot(ll_vec_euc)
plt.xlabel('iter')
plt.ylabel('log-likelihood')
plt.grid()
plt.title("log-likelihood")
plt.legend(["hyperbolic", "euclidean"])
plt.savefig(f"results/M{number}/ll_plot_M{number}.png") 

### Euclidean

In [ ]:
args.args

In [ ]:
n_leaves = 20
tree, opts = randomt_tree(n_leaves)
evo_model = JukesCantor(alpha=0.1)
sim_seq = simulate_seq_all(tree, evo_model, ndata=len(tree))

tree, ll_vec = train_loop2(sim_seq, evo_model)

In [ ]:
plt.plot(ll_vec_euc)
plt.xlabel('iter')
plt.ylabel('log-likelihood')
plt.title("Euclidean log-likelihood")
plt.grid()
plt.xlabel("iter")
plt.ylabel("lig-likelihood")

In [88]:
# convert to fasta
def save_as_fasta(sequences, file_name):
    with open(file_name, 'w') as f:
        for i, seq in enumerate(sequences):
            # Write the header line
            f.write(f">Sequence_{i+1}\n")
            # Write the sequence line (convert array to a single string)
            f.write("".join(seq) + "\n")

# Save the NumPy array as a FASTA file
save_as_fasta(sim_seq, "euc_simulated_sequences.fasta")


import pickle
with open("euc_predicted_tree.pkl", "wb") as f:
    pickle.dump(tree, f)

# save log-likelihood
np.savetxt('euc_ll_vec.txt',ll_vec)

Another Method

In [ ]:
from utils.tree.sem import randomt_tree, main

tree, opts, leaf_sequences, ll_vec_init = main(n_leaves=20, ndata=39)

# encode the leaf sequences
encoded_seq = one_hot(leaf_sequences)
leaf_features = np.argmax(encoded_seq, axis=-1)

n_leaves, n_sites = leaf_sequences.shape
# construct a initial random tree ontology
# tree, opts = randomt_tree(n_leaves)

leaves = [n for n in tree if tree.nodes[n]['type'] == 'leaf']
root = len(tree) - 1    

sigma_l = 0.1  # annealing std
rho = 0.95  # cooling factor
max_iter = 100
T_l = tree  # init topology
ll_vec = []  # log-likelihood
Tl_vec=[]
max_iter = 100

print("start training loop")
for iter in range(max_iter):
    print(f'start iter: {iter}')
    
    Tl_vec.append(tree)
    up_table = cmp.compute_up_messages(leaf_sequences, tree, evo_model)
    down_table = cmp.compute_down_messages(leaf_sequences, tree, evo_model, up_table)
    # compute log-likelihood
    ll_sites, ll = cmp.compute_loglikelihood(up_table, evo_model.stat_prob)
    if len(ll_vec)>0 and ll_vec[0]>ll:#figure out argmax maybe
        tree = Tl_vec[0]
        ll_vec.append(ll_vec[0])
        continue
    ll_vec.append(ll)
    print(f'log-likelihood: {ll}')

    W, opts = cmp.compute_weight_matrix(tree, evo_model, up_table, down_table)
    # print("computed weighted matrix")

    # generate interior nodes and (unweighted) adj matrix sequences given leaf sequences
    features, adj_matrix = generate_interior_node(tree, leaf_features, evo_model)
    print("generated interior nodes")
    
    # train 
    adj = sp.csr_matrix(adj_matrix) # scipy sparce matrix
    labels = get_labels(tree)
    data = load_inference_data(adj, features, labels, opts, args.args)
    print("start training")
    weighted_adj_matrix, labels, opts, model = train.train(args.args, dat, n_features)
    print("finished training")
    # construct mst from weighted adj matrix
    mst = get_mst(weighted_adj_matrix, opts)
    # construct bifurcate tree from mst, as the tree onto for next iter
    tree, edge = bifurcate_mst(mst, leaves, root)

    sigma_l *= rho
    if sigma_l <= 5e-3:
        break
    elif iter > 0 and np.abs(ll_vec[iter] - ll_vec[iter - 1]) < 1e-3:
        break

    print(f'finished iter: {iter}')


## Archive

#### 1. On Training Data

In [ ]:
save_path = '/Users/liubojun/Desktop/hgcn/logs/lp/2024_11_20/1/'
adj_matrix = np.load(os.path.join(save_path,"adj_matrix.npy"), allow_pickle=True)
adj_matrix = adj_matrix.item() 
# adj_matrix = (adj_matrix.item()).toarray()
features = np.load(os.path.join(save_path,"feats.npz"))
features = features['features']

embeddings = np.load(os.path.join(save_path,"embeddings.npy"))
labels = np.loadtxt(os.path.join(save_path, "labels.txt"))
opt = np.loadtxt(os.path.join(save_path, "opt.txt"))

arg_path = os.path.join(save_path,"args.pkl")
print(arg_path)
with open(arg_path, 'rb') as f:
    args = pickle.load(f)

In [ ]:
# only for traing data
embeddings.shape

In [ ]:
import torch
from models.base_models import NCModel, LPModel

# Instantiate the model (you need to use the same architecture that was trained)
Model = LPModel(args)  # Replace with the appropriate model class

# Load the trained model parameters
Model.load_state_dict(torch.load(model_dir))
Model.eval()  # Set the model to evaluation mode

In [ ]:
from utils.tree.load_parameters import generate_data
%load_ext autoreload
%autoreload 2

In [ ]:
# Convert feature matrix to a PyTorch tensor
feature_tensor = torch.tensor(features, dtype=torch.float)
print(feature_tensor.shape)
print(adj_matrix.shape)

# Convert adjacency matrix to a PyTorch sparse tensor
adj_coo = adj_matrix.tocoo()
indices = torch.from_numpy(np.vstack((adj_coo.row, adj_coo.col)).astype(np.int64))
print(indices.shape)

values = torch.from_numpy((adj_coo.data).astype(np.float32))
print(values.dtype)

shape = torch.Size(adj_coo.shape)
adj_tensor = torch.sparse.FloatTensor(indices, values, shape)
print(adj_tensor.dtype)

# embeddings_tensor = torch.tensor(embeddings, dtype=torch.float)
# print(embeddings_tensor.shape)

# Generate node embeddings using the model's `encode()` function
with torch.no_grad():
    embeddings = Model.encode(feature_tensor, adj_tensor)
    link_probabilities = Model.decode(embeddings, indices.T)

In [ ]:
link_probabilities.shape

In [59]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [ ]:
indices_np = indices.cpu().numpy().T  # Shape: (n_edges, 2)
link_probabilities_np = link_probabilities.cpu().numpy()  # Shape: (n_edges,)

# Create a list of tuples to store node pairs and their corresponding probabilities
link_predictions = []
count = 0
for i in range(len(indices_np)):
    node_pair = (indices_np[i][0], indices_np[i][1])  # Extract node pair (u, v)
    probability = link_probabilities_np[i]  # Extract predicted probability
    link_predictions.append((node_pair[0], node_pair[1], probability))
    if probability >  0.00001:
        count += 1
print(count)

# Print the matched node pairs and link probabilities
for node1, node2, prob in link_predictions:
    print(f"Node Pair ({node1}, {node2}): Link Probability = {prob:.4f}")

In [ ]:
adj_matrix.shape

In [ ]:
link_pred_np = np.array(link_predictions)
# Initialize an n_nodes x n_nodes adjacency matrix with zeros
n_nodes = adj_matrix.shape[0]
weighted_adj_matrix = np.zeros((n_nodes, n_nodes))
# Populate the adjacency matrix based on link_predictions
for i in range(link_pred_np.shape[0]):
    u, v, weight = int(link_pred_np[i, 0]), int(link_pred_np[i, 1]), link_pred_np[i, 2]
    weighted_adj_matrix[u, v] = weight
    weighted_adj_matrix[v, u] = weight  # Assuming the graph is undirected

# Print the resulting weighted adjacency matrix
print("Weighted Adjacency Matrix:")
print(weighted_adj_matrix[100][102])

In [ ]:
link_pred_np

In [ ]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [86]:
import networkx as nx
def plot_graph(graph):
    # Generate layout for the graph nodes
    pos = nx.spring_layout(graph)  # You can also use other layouts like nx.kamada_kawai_layout(graph)

    # Draw the nodes
    nx.draw_networkx_nodes(graph, pos, node_size=50)

    # Draw the edges
    nx.draw_networkx_edges(graph, pos, edgelist=graph.edges(), width=1)

    # Draw the labels for nodes and edges
    nx.draw_networkx_labels(graph, pos, font_size=5, font_family='sans-serif')
    edge_labels = nx.get_edge_attributes(graph, 'weight')
    # nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

    # Display the graph
    plt.title("Tree")
    plt.show()

In [ ]:
from utils.tree.mst_utils import bifurcate_mst, get_mst
%load_ext autoreload
%autoreload 2

In [ ]:
mst = get_mst(weighted_adj_matrix, opt)
plot_graph(mst)

In [ ]:
root = [node for node in mst.nodes if labels[node] == 0]
leaves = [node for node in mst.nodes if labels[node] == 1]

bifurcate_tree, edge = bifurcate_mst(mst, leaves, 198)

bifurcate_tree

In [ ]:
plot_graph(bifurcate_tree)

#### 2. On Simulated Data

In [ ]:
from utils.tree.load_parameters import generate_data
%load_ext autoreload
%autoreload 2

adj_matrix, features, labels = generate_data(n_leaves=100, alpha=0.1)

In [ ]:
# Convert feature matrix to a PyTorch tensor
feature_tensor = torch.tensor(features, dtype=torch.float)
print(feature_tensor.shape)

# Convert adjacency matrix to a PyTorch sparse tensor
adj_coo = adj_matrix.tocoo()
indices = torch.from_numpy(np.vstack((adj_coo.row, adj_coo.col)).astype(np.int64))
print(indices.dtype)

values = torch.from_numpy((adj_coo.data).astype(np.float32))
print(values.dtype)

shape = torch.Size(adj_coo.shape)
adj_tensor = torch.sparse.FloatTensor(indices, values, shape)
print(adj_tensor.dtype)

embeddings_tensor = torch.tensor(embeddings, dtype=torch.float)

# Generate node embeddings using the model's `encode()` function
with torch.no_grad():
    link_probabilities = Model.decode(embeddings_tensor, indices.T)


In [226]:
positive_pairs = np.array(sp.find(adj_matrix)[:2]).T

n_nodes = adj_matrix.shape[0]  # Number of nodes

# Create a set of all possible pairs of nodes
all_pairs = set((i, j) for i in range(n_nodes) for j in range(i + 1, n_nodes))

# Create a set of all positive pairs
positive_pairs_set = set(tuple(pair) for pair in positive_pairs)

# Generate negative pairs by taking the difference between all possible pairs and positive pairs
negative_pairs_set = all_pairs - positive_pairs_set

# Convert the negative pairs set to a numpy array
negative_pairs = np.array(list(negative_pairs_set))

In [ ]:
positive_labels = np.ones(len(positive_pairs))
negative_labels = np.zeros(len(negative_pairs))

# Combine pairs and labels
node_pairs = np.vstack([positive_pairs, negative_pairs])
labels = np.hstack([positive_labels, negative_labels])

len(node_pairs), len(labels)

In [ ]:
embeddings_tensor = torch.tensor(embeddings, dtype=torch.float)
node_pairs_tensor = torch.tensor(node_pairs, dtype=torch.long)

u_embeddings = embeddings_tensor[node_pairs_tensor[:, 0]]  # Embeddings for the first node in each pair
v_embeddings = embeddings_tensor[node_pairs_tensor[:, 1]]  # Embeddings for the second node in each pair
scores = torch.sum(u_embeddings * v_embeddings, dim=1)

threshold = 0.9  # Adjust as needed
predicted_links = scores > threshold

len(predicted_links==1)

(predicted_links==True).sum()

Visualize Predicted Links on Graph

In [88]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Create an empty graph
G = nx.Graph()  # Use nx.DiGraph() if you want a directed graph

# Add nodes (optional, NetworkX will add nodes automatically when adding edges)
nodes = np.unique(node_pairs)  # Get unique nodes
G.add_nodes_from(nodes)

# Add edges based on predicted links
for i, (u, v) in enumerate(node_pairs):
    if predicted_links[i]:
        G.add_edge(u, v, weight=scores[i])

In [ ]:
# Set the layout for the graph (e.g., spring_layout, circular_layout, random_layout)
pos = nx.spring_layout(G)  # Positions the nodes for visualization

# Draw the graph
plt.figure(figsize=(10, 8))
nx.draw(
    G, pos,
    with_labels=True,
    node_color='skyblue',
    edge_color='gray',
    node_size=100,
    font_size=10,
    font_weight='bold'
)

plt.title("Predicted Graph Visualization")
plt.show()


Maximum Spanning Tree

In [90]:
# Create a new graph with negated weights
G_neg = nx.Graph()
for u, v, data in G.edges(data=True):
    G_neg.add_edge(u, v, weight=-data['weight'])  # Negate the weights

# Use NetworkX to find the minimum spanning tree (which, with negated weights, is equivalent to MST)
mst_neg = nx.minimum_spanning_tree(G_neg)

# Convert the MST back to positive weights for the actual Maximum Spanning Tree
mst = nx.Graph()
for u, v, data in mst_neg.edges(data=True):
    mst.add_edge(u, v, weight=-data['weight'])  # Revert to original weights


In [ ]:
import matplotlib.pyplot as plt

# Set the layout for visualization
pos = nx.spring_layout(mst)  # Positioning the nodes for visualization

# Draw the MST
plt.figure(figsize=(10, 8))
nx.draw(
    mst, pos,
    with_labels=True,
    node_color='lightblue',
    edge_color='gray',
    node_size=800,
    font_size=10,
    font_weight='bold'
)

# Draw edge labels with weights
edge_labels = nx.get_edge_attributes(mst, 'weight')
nx.draw_networkx_edge_labels(mst, pos, edge_labels=edge_labels)

plt.title("Maximum Spanning Tree Visualization")
plt.show()
